In [1]:
from kingdon import Algebra
from sympy import ccode

In [2]:
alg = Algebra(2, 0, 1)

In [3]:
a = alg.multivector(name="a")
b = alg.multivector(name="b")

In [4]:
alg.bin2canon

{0: 'e', 1: 'e0', 2: 'e1', 3: 'e01', 4: 'e2', 5: 'e02', 6: 'e12', 7: 'e012'}

In [5]:
r = a * b

In [6]:
code = ccode(r.values()[0])
code

'a*b + a1*b1 - a12*b12 + a2*b2'

In [7]:
code.replace("a", "a.e").replace("b", "b.e")

'a.e*b.e + a.e1*b.e1 - a.e12*b.e12 + a.e2*b.e2'

In [16]:
# Convert multivector to C++ macro code
def mv_to_code(mv, name, op="+="):
    lines = []
    for key, value in zip(mv.keys(), mv.values()):
        key = "    r." + alg.bin2canon[key]
        value = ccode(value).replace("a", "p_a.e").replace("b", "p_b.e")
        line = key + f" {op} " + value + ";"
        lines.append(line)
    code = "\\\n".join(lines)
    code = "#define {} \\\n".format(name) + code
    return code

fragment = mv_to_code(r, "GP_full")
print(fragment)


#define GP_full \
    r.e += p_a.e*p_b.e + p_a.e1*p_b.e1 - p_a.e12*p_b.e12 + p_a.e2*p_b.e2;\
    r.e0 += p_a.e*p_b.e0 + p_a.e0*p_b.e + p_a.e01*p_b.e1 - p_a.e012*p_b.e12 + p_a.e02*p_b.e2 - p_a.e1*p_b.e01 - p_a.e12*p_b.e012 - p_a.e2*p_b.e02;\
    r.e1 += p_a.e*p_b.e1 + p_a.e1*p_b.e + p_a.e12*p_b.e2 - p_a.e2*p_b.e12;\
    r.e2 += p_a.e*p_b.e2 + p_a.e1*p_b.e12 - p_a.e12*p_b.e1 + p_a.e2*p_b.e;\
    r.e01 += p_a.e*p_b.e01 + p_a.e0*p_b.e1 + p_a.e01*p_b.e + p_a.e012*p_b.e2 - p_a.e02*p_b.e12 - p_a.e1*p_b.e0 + p_a.e12*p_b.e02 + p_a.e2*p_b.e012;\
    r.e02 += p_a.e*p_b.e02 + p_a.e0*p_b.e2 + p_a.e01*p_b.e12 - p_a.e012*p_b.e1 + p_a.e02*p_b.e - p_a.e1*p_b.e012 - p_a.e12*p_b.e01 - p_a.e2*p_b.e0;\
    r.e12 += p_a.e*p_b.e12 + p_a.e1*p_b.e2 + p_a.e12*p_b.e - p_a.e2*p_b.e1;\
    r.e012 += p_a.e*p_b.e012 + p_a.e0*p_b.e12 + p_a.e01*p_b.e2 + p_a.e012*p_b.e - p_a.e02*p_b.e1 - p_a.e1*p_b.e02 + p_a.e12*p_b.e0 + p_a.e2*p_b.e01;


In [30]:
def generate_binary_op(name="GP", op=lambda a, b: a * b):
    frags = []
    for i in range(4):
        for j in range(4):
            a = alg.multivector(name="a", grades=(i,))
            b = alg.multivector(name="b", grades=(j,))
            r = op(a, b)
            frag = mv_to_code(r, "{}_{}_{}".format(name, i, j), "+=")
            frags.append(frag)

    return frags

def generate_unary_op(name="REVERSE", op=lambda a: a.reverse()):
    frags = []
    for i in range(4):
        a = alg.multivector(name="a", grades=(i,))
        r = op(a)
        frag = mv_to_code(r, "{}_{}".format(name, i), "=")
        frags.append(frag)

    return frags



def generate_fragments():
    frags = []

    # NO NECESSARY
    # Geometric Product in full
    # Outer Product in full
    # Inner Product in full
    # Regressive Product in full

    # TOO EXPENSIVE
    # Sandwitch(Conjugate) Product in full
    # Project in full

    # Geometric Product in grades
    frags += generate_binary_op(name="GP", op=lambda a, b: a * b)

    # Outer Product in grades
    frags += generate_binary_op(name="OP", op=lambda a, b: a ^ b)
    
    # Inner Product in grades
    frags += generate_binary_op(name="IP", op=lambda a, b: a | b)

    # Regressive Product in grades
    # TODO: check is RP splitable
    frags += generate_binary_op(name="RP", op=lambda a, b: a & b)
    
    # Commutator Product
    frags += generate_binary_op(name="CP", op=lambda a, b: a.cp(b))

    # Reverse
    frags += generate_unary_op(name="REVERSE", op=lambda a: a.reverse())
    # Dual
    frags += generate_unary_op(name="DUAL", op=lambda a: a.dual())
    
    # Undual, the same with Dual

    return "\n\n".join(frags)

header = """// Generate from https://github.com/xiongyaohua/PGA_codegen

"""
code = header + generate_fragments()
with open("pga2.inc", "w") as f:
    f.write(code)